# Flow Modell für zweidimensionale Datensätze

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import os
import random
from PIL import Image

#Kleine Module von Lukas Rinder https://github.com/LukasRinder/normalizing-flows:
from LukasRinder.LukasRinder import Made
from LukasRinder.LukasRinder import train_density_estimation, nll
from LukasRinder.LukasRinder import plot_heatmap_2d

tfd = tfp.distributions
tfb = tfp.bijectors

tf.random.set_seed(1234)

### Nicht alle Versionen von Tensorflow-Probability sind mit alles Python Versionen kompatibel. Tfp steckt noch in der Beta-Version.

In [ ]:
print(np.__version__, tf.__version__, tfp.__version__, sep="\n")

# Goethe Daten mit Verwerfungsmethode generieren.

### Bild des Goethe Universitäts Logos laden.

In [ ]:
img = Image.open("goethe-kopf.jpg")
img = img.convert("1")
img1 = np.asarray(img)
epsilon = 1

### Zufallszahlen erzeugen.

In [ ]:
y_cords = np.random.randint(low=epsilon, high=1497-epsilon, size=1000000)
x_cords = np.random.randint(low=epsilon, high=1663-epsilon, size=1000000)

### Nur Daten, die maximal Epsilon Pixel (in Maximumsnorm) Abstand zu farbigen Pixeln des Logos haben, behalten.

In [ ]:
x_data, y_data = [], []
for i in range(1000000):
    keep = False
    for j in range(2*epsilon):
        for k in range(2*epsilon):
            if img1[y_cords[i]+j-epsilon][x_cords[i]+k-epsilon] == False:
                keep = True
    if keep == True:
        x_data.append(x_cords[i])
        y_data.append(y_cords[i])

### Y Koordinaten spiegeln. 
#### Dies ist nötig, da jpg Dateien ungewöhnliche koordinatenachsen nutzen.

In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)
y_data = np.where(y_data > ((1493-epsilon)/2), y_data - 2*(y_data - (1493-epsilon)/2), y_data + 2*((1493-epsilon)/2 - y_data))

### Alle Daten visualisieren

In [ ]:
plt.scatter(x_data, y_data, s=1)

### Daten auf [0,1]^2 skalieren.

In [ ]:
x_data = np.subtract(x_data, min(x_data)*np.ones(x_data.shape))
x_data = x_data/max(x_data)
y_data = np.subtract(y_data, min(y_data)*np.ones(y_data.shape))
y_data = y_data/max(y_data)

### Daten konkatenieren.

In [ ]:
data = []
for i in range(len(x_data)):
    data.append(np.array([x_data[i], y_data[i]]))
data = np.array(data)

In [ ]:
x_train = tf.cast(data[:200000], tf.float32)
x_val = tf.cast(data[200000:300000], tf.float32)

### Batches initialisieren.

In [ ]:
batch_s = 128

x_train = tf.cast(data[:50000], tf.float32)
x_val = tf.cast(data[50000:], tf.float32)
batched_train = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000).batch(batch_s)
batched_val = tf.data.Dataset.from_tensor_slices(x_val).batch(batch_s)

### Funktion, die ein MAF bzw. IAF Modell erzeugt.
#### Die Permutation ist hier fest gewählt und vertauscht die Einträge. Die einzelnen Transformationen werden mit tfb.Chain verkettet. Hier wird das in umgekehrter Reihenfolge getan, sodass die zuerst implementierte Transformation T1 entspricht (auf dem latenten Raum operiert).

In [ ]:
def AutoregressiveFlow_2D(layers, hidden_shape=[64, 64], activation="relu", inverse=False):
    base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros(shape=[2], dtype=tf.float32))
    bijectors = []
    params=0
    if inverse:
        for i in range(layers):
            bijectors.append(tfb.Invert(tfb.MaskedAutoregressiveFlow(
                shift_and_log_scale_fn = Made(params=2, hidden_units=hidden_shape, activation=activation))))
            bijectors.append(tfb.Permute(permutation=[1, 0]))
        bijector = tfb.Chain(bijectors=list(reversed(bijectors)))
    else:
        for i in range(layers):
            bijectors.append(tfb.MaskedAutoregressiveFlow(
                shift_and_log_scale_fn = Made(params=2, hidden_units=hidden_shape, activation=activation)))
            bijectors.append(tfb.Permute(permutation=[1, 0]))
        bijector = tfb.Chain(bijectors=list(reversed(bijectors)))

    masked_auto_flow = tfd.TransformedDistribution(distribution=base_dist, bijector=bijector)
    masked_auto_flow.log_prob(base_dist.sample())
    for theta in masked_auto_flow.trainable_variables:
        params += np.prod(theta.shape)
    print("trainable parameters:", params)
    return masked_auto_flow, base_dist, bijectors

### Parameter festlegen und einen Namen für die Checkpoints festlegen.

In [ ]:
layers = 60
base_lr = 1e-3
end_lr = 1e-4
epochs = 200
dataset = "goethe"
trainsize = 50000

### Modell initialisieren. In diesem Stadium entspricht MAF der Startverteilung bzw. full_bijector der Identitätsabbildung.

In [ ]:
MAF, base_dist, list_of_bijectors = AutoregressiveFlow_2D(layers, inverse=False)

In [ ]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(base_lr, epochs, end_lr, power=0.5)
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### Checkpoints initialisieren.

In [ ]:
ckpt_dir = f"{dataset}/tmp_{layers}"
ckpt_prefix = os.path.join(ckpt_dir, "ckpt")

ckpt = tf.train.Checkpoint(optimizer=opt, model=MAF)

### Funktion, die ein Modell trainiert.
#### Dabei werden Trainings- und Validierungsdaten verwendet, um Overfitting festzustellen. Nach dem Durchlaufen aller Epochen, wird die benötigte Zeit ausgegeben. Zudem kann gewählt werden, ob die Dichte während des Trainings visualisiert werden soll.

In [ ]:
def TrainFlow(flow, batched_train, batched_val, epochs, train_size, optimizer, checkpoint, checkpoint_pref, heat_name=None):

    t_losses, v_losses = [], []
    t_start = time.time()
    
    for i in range(epochs):
        batched_train.shuffle(buffer_size=train_size, reshuffle_each_iteration=True)
        batch_t_losses = []
        for batch in batched_train:
            batch_loss = train_density_estimation(flow, optimizer, batch)
            batch_t_losses.append(batch_loss)
        t_loss = tf.reduce_mean(batch_t_losses)

        batch_v_losses = []
        for batch in batched_val:
            batch_loss = nll(flow, batch)
            batch_v_losses.append(batch_loss)
        v_loss = tf.reduce_mean(batch_v_losses)

        t_losses.append(t_loss)
        v_losses.append(v_loss)
        print(f"Epoch {i+1}: train loss: {t_loss}, val loss: {v_loss}")
        
        if i == 0:
            min_v_loss = v_loss
            best_epoch = 0
        if v_loss < min_v_loss:
            min_v_loss = v_loss
            best_epoch = i
            checkpoint.write(file_prefix=checkpoint_pref)
            
        if heat_name:        
            if (i < 12) or (i % 30 == 0):
                plot_heatmap_2d(flow, 0.0, 1.0, 0.0, 1.0, mesh_count=500, name=heat_name + str(i+1))
                
    print("train time:", time.time() - t_start)
    
    return t_losses, v_losses

In [ ]:
train_losses, val_losses = TrainFlow(MAF, batched_train, batched_val, 
                                     epochs, trainsize, opt, ckpt, ckpt_prefix, heat_name="goethe_training")

### Plot der Verluste während des Trainings.

In [ ]:
plt.plot(range(len(train_losses)), train_losses, label="train loss")
plt.plot(range(len(val_losses)), val_losses, label="val loss")
plt.legend()

### Laden des Stadiums des Modelles mit geringstem Verlust auf den Validierungsdaten.

In [ ]:
ckpt.restore(ckpt_prefix)

### Neue Daten generieren.

In [ ]:
samples = MAF.sample(5000).numpy()
fig = plt.figure(figsize=(4, 4), dpi=80)
plt.scatter(samples[:,0], samples[:,1], s=1)
plt.savefig(f"{layers}_layers_{epochs}_epochs_{dataset}.png")

### Zwei Funktionen, die zusammen die gesammte Transformation Schrittweise darstellen. 
#### Immer zwei aufeinander folgende Schritte sind von gleicher Orientierung, dann wird an der Winkelhalbierenden gespiegelt (Permutation).

In [ ]:
def PlotStep(points, save=False, name=None):
    points = points.numpy()
    fig = plt.figure(figsize=(4, 4), dpi=80)
    plt.scatter(points[:,0], points[:,1], s=1)
    if save:
        plt.savefig(name+".png")

In [ ]:
def FlowSteps(distribution, bijectors_list, samples, steps="all", save=False, name="empty"):
    points = distribution.sample(samples)
    PlotStep(points, save=save, name=name+"0")
    last = True

    if steps == "all":
        for bijector in bijectors_list:
            points = bijector.forward(points)
            PlotStep(points)
            last = False

    if steps != "all":
        stepsize = len(bijectors_list) // steps
        points = bijectors_list[0].forward(points)
        PlotStep(points)
        counter = 1
        while counter < (len(bijectors_list) - stepsize + 1):
            for i in range(stepsize):
                points = bijectors_list[counter+i].forward(points)
            PlotStep(points, save=save, name=name+str(counter))
            counter += stepsize
            if counter == len(bijectors_list) - 1:
                last = False
    if last:
        while counter < len(bijectors_list):
            points = bijectors_list[counter].forward(points)
            counter += 1
        PlotStep(points, save=save, name=name+str(len(bijectors_list)))

In [ ]:
FlowSteps(base_dist, list_of_bijectors, 8000)

### Darstellung der erzeugten Dichte des Modells.

In [ ]:
plot_heatmap_2d(MAF, 0.0, 1.0, 0.0, 1.0, mesh_count=1000)

### Durchschnittlich benötigte Zeit zum Generieren einer Stichprobe.

In [ ]:
time_s = time.time()
MAF.sample(2000000)
av_sample_time = (time.time() -time_s)/2000000
print(av_sample_time)